# Setting Portion

In [1]:
#Packages importing
import ee
import geemap
import multiprocess as mp
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
pd.set_option('display.precision', 20)
import geopandas as gpd
import json
import shapely.geometry as sg
import random
from shapely.geometry import Polygon

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
# Load the image collection.
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2021-01-01', '2024-12-31'))
#Fixed Parameters
# Visualization parameters.
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
    'gamma': [1.0, 1.0, 1.0]
}

# Code Portion

### display_masked_image

In [57]:
def display_masked_image(bounding_box_coords,houseCoords,name):

    min_lon, min_lat, max_lon, max_lat = bounding_box_coords
    boundingBox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])
    #boundingBox = ee.Geometry.Polygon(obb_coords)  

    # Create an image composite or use the first image.
    firstImage = dataset.median()
   
    # Clip the image to the bounding box.
    clippedImage = firstImage.clip(boundingBox)
   
    # Create a mask from the houseCoords polygon.
    mask = ee.Image.constant(1).clip(houseCoords).mask()
   
    # Apply the mask to the clipped image.
    blackImage = ee.Image.constant([0, 0, 0, 255]).clip(boundingBox)
    maskedImage = clippedImage.updateMask(mask).blend(blackImage.updateMask(mask.Not()))
    # Initialize a map.
    m = geemap.Map()
    m.set_center((max_lon + min_lon) / 2, (max_lat + min_lat) / 2, 20)  # Center the map on the bounding box.
    m.add_layer(maskedImage, trueColorVis, 'True Color')
   
    visualizedImage = maskedImage.visualize(**trueColorVis)
    #geemap.ee.batch.Export.image.toDrive(visualizedImage, description="testingexample"+str(name), folder="QTM550Team1Plottings1", region=boundingBox,skipEmptyTiles=True, scale=0.05)
    geemap.ee_export_image_to_drive(visualizedImage, description="testingexample"+str(name), folder="QTM550Team1Plottings", region=boundingBox, scale=0.05)

    # Display the map.


### extract_polygon_coords

In [4]:
def extract_polygon_coords(polygon):
    if polygon.geom_type == 'Polygon':
        return [list(polygon.exterior.coords)]
    elif polygon.geom_type == 'MultiPolygon':
        return [list(poly.exterior.coords) for poly in polygon.geoms]

###  run_function

In [60]:
#the index we need to write is the serial number of the file
def run_function(index):
    Splitdf   = gpd.read_file( mypath +'/Split_GeoJson'+  str(index) + ".geojson")
    Splitdf['ParcelID1'] = Splitdf['ParcelID'].str.replace(' ', '')
    geometry = Splitdf['geometry']
    bounds=geometry.bounds
    bounds['polygon'] =geometry.apply(extract_polygon_coords)

    for i in range(0,2):
    #for i in range(len(Splitdf)):
        bounding_box_coords  = (bounds.minx.values[i], bounds.miny.values[i], bounds.maxx.values[i], bounds.maxy.values[i])
        houseCoords = ee.Geometry.Polygon(bounds['polygon'][i])

        #polygon = sg.Polygon(geometry[i])
        #obb = polygon.minimum_rotated_rectangle
        #obb_coords = list(obb.exterior.coords)
        # Replace spaces with nothing
        name=Splitdf['ParcelID1'][i]
        #display_masked_image(obb_coords, houseCoords,name)
        display_masked_image(bounding_box_coords, houseCoords,name)
        tasks = ee.batch.Task.list()
        # Print details of all tasks, looking for your specific task by description.
        for task in tasks:
            if task.config['description'] == 'testingexample':
                print(task.status())
        


### Operation Area

In [61]:
mypath="/Users/gehaojie/Documents/GitHub/QTM550/Split"
run_function('1')

### Testing Area()

In [55]:
Splitdf1=gpd.read_file("/Users/gehaojie/Documents/GitHub/QTM550/Split/Split_GeoJson5.geojson")
Splitdf1['ParcelID1'] = Splitdf1['ParcelID'].str.replace(' ', '')
geometry1 = Splitdf1['geometry']
bounds1=geometry1.bounds
bounds1['polygon'] =geometry1.apply(extract_polygon_coords)

    #for i in range(len(Splitdf)):
        #min_lon, min_lat, max_lon, max_lat = bounds.minx.values[i], bounds.miny.values[i], bounds.maxx.values[i], bounds.maxy.values[i]
i=40
bounding_box_coords1  = (bounds1.minx.values[i], bounds1.miny.values[i], bounds1.maxx.values[i], bounds1.maxy.values[i])
houseCoords1 = ee.Geometry.Polygon(bounds1['polygon'][i])
        # Replace spaces with nothing
name1=Splitdf1['ParcelID1'][i]


polygon = sg.Polygon(geometry1[i])
obb = polygon.minimum_rotated_rectangle
# Convert the coordinates into an Earth Engine polygon geometry
obb_coords = list(obb.exterior.coords)

# Iterate over each row in the Series
#for index, row in bounds1['polygon'].iteritems():
    # Convert the row to a list of lists (each inner list is a coordinate pair)
#    coords = [list(coord) for coord in row[0]] 

#houseCoords1 = ee.Geometry.Polygon(coords)

#display(geometry1)
#display(coords)
#geometry1[10]

In [53]:
bounding_box_coords1

(-84.2617152998281, 34.1229577870579, -84.2615896642103, 34.1230210901029)

In [56]:
import ee
import geemap
min_lon, min_lat, max_lon, max_lat = bounding_box_coords1
   
    # Create a bounding box for the area of interest.
boundingBox1 = ee.Geometry.Polygon(obb_coords)

    # Create an image composite or use the first image.
firstImage = dataset.median()
   
    # Clip the image to the bounding box.
clippedImage = firstImage.clip(boundingBox1)
   
    # Create a mask from the houseCoords polygon.
mask = ee.Image.constant(1).clip(houseCoords1).mask()

# Apply the mask to the clipped image.
#maskedImage = clippedImage.updateMask(mask)
blackImage = ee.Image.constant([0, 0, 0, 255]).clip(boundingBox1)
maskedImage = clippedImage.updateMask(mask).blend(blackImage.updateMask(mask.Not()))
# Visualization parameters.
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
}

# Initialize a map.
m = geemap.Map()
m.set_center((max_lon + min_lon) / 2, (max_lat + min_lat) / 2, 20)  # Center the map on the bounding box.
m.add_layer(maskedImage, trueColorVis, 'True Color')

# Display the map.
m

#name=1234
#visualizedImage = maskedImage.visualize(**trueColorVis)
#geemap.ee_export_image_to_drive(visualizedImage, description="testingexample"+str(name), folder="QTM550Team1Plottings", region=boundingBox1, scale=10**-6)


Map(center=[34.122027192499246, -84.26044433988035], controls=(WidgetControl(options=['position', 'transparent…

In [104]:
import ee
import geemap

# Initialize the Earth Engine library.
ee.Initialize()

# Load the image collection.
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2021-01-01', '2023-12-31'))


# Define the bounding box coordinates for the broader area.
max_lon, max_lat, min_lon, min_lat = -84.321957, 34.078049, -84.322475, 34.077582

# Create a bounding box for the area of interest (around your house).
boundingBox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Define the coordinates of your land.
houseCoords = ee.Geometry.Polygon(
 [(-84.3219570886809, 34.0776338049394), (-84.3222598355553, 34.0775826902571), (-84.3222644330423, 34.0775821533882), (-84.3222706029606, 34.0775817479663), (-84.3222752446441, 34.0775816789761), (-84.3222814287089, 34.0775818992524), (-84.3222875798376, 34.0775824766072), (-84.3222921535182, 34.0775831416122), (-84.3222981746658, 34.0775843340284), (-84.3223026169916, 34.0775854554135), (-84.3223084203637, 34.0775872467734), (-84.3223154421995, 34.0775899499196), (-84.3223208395029, 34.0775924711948), (-84.322327270254, 34.0775960496498), (-84.3223333030951, 34.0776000794129), (-84.3223378137524, 34.0776036071046), (-84.3223420182325, 34.0776073858313), (-84.3223458980858, 34.0776113993874), (-84.322350262488, 34.0776167142529), (-84.3223533446141, 34.0776211805255), (-84.3223610479689, 34.0776345308077), (-84.3223668920169, 34.077644337417), (-84.3223729385296, 34.0776540590183), (-84.3223791842515, 34.0776636929107), (-84.322385629176, 34.0776732354879), (-84.3223922711311, 34.0776826840477), (-84.3223991068632, 34.077692036791), (-84.3224061352803, 34.0777012892112), (-84.322413354214, 34.0777104404092), (-84.3224207614904, 34.0777194867815), (-84.3224283549362, 34.0777284247242), (-84.322436131299, 34.07773725334), (-84.3224440905723, 34.0777459690222), (-84.3224522284187, 34.0777545699732), (-84.3224605426629, 34.0777630516876), (-84.3224690322215, 34.0777714141668), (-84.322475290727, 34.0777773676702), (-84.3221294753502, 34.0780105586958), (-84.3220723494629, 34.0780490458033), (-84.3219570886809, 34.0776338049394)]
)

# Create an image composite or use the first image.
firstImage = dataset.median()

# Clip the image to the bounding box.
clippedImage = firstImage.clip(boundingBox)

# Create a mask from the houseCoords polygon.
mask = ee.Image.constant(1).clip(houseCoords).mask()

# Apply the mask to the clipped image.
maskedImage = clippedImage.updateMask(mask)


# Visualization parameters.
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
}

# Initialize a map.
m = geemap.Map()
m.set_center((max_lon + min_lon) / 2, (max_lat + min_lat) / 2, 20)  # Center the map on the bounding box.
m.add_layer(maskedImage, trueColorVis, 'True Color')

# Display the map.
m


Map(center=[34.0778155, -84.322216], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…